In [25]:
from dotenv import dotenv_values
config = dotenv_values(".env")

In [66]:
import pandas as pd
import psycopg2
conn = psycopg2.connect(dbname=config['DBNAME'], user=config['USER'], password=config['PASSWORD'])

In [67]:
# Get last date tables
df_tables = pd.read_sql('SELECT table_name FROM information_schema.tables WHERE table_schema=\'' + config['SCHEMA'] +'\'   AND table_type=\'BASE TABLE\';', conn)
df_tables

,table_name
0,morticd_7_22b4_1642928616
1,morticd_8_9366_1642928616
2,morticd_9_71c9_1642928616
3,morticd_10_1_f695_1642928617
4,morticd_10_2_ffb1_1642928617
5,morticd_10_3_1c6a_1642928618
6,morticd_10_4_453e_1642928618
7,countrycodes_0_8c41_1642928619
8,notes_0_a67b_1642928619
9,population_0_3b8f_1642928619


In [68]:
max_timestamp = df_tables['table_name'].apply(lambda x: x[x.rfind('_') +1 :-2]).max()
max_timestamp


'16429286'

In [83]:
df_lastupdate_tables = df_tables['table_name'].apply(lambda x: max_timestamp in x)
df_tables[df_lastupdate_tables]

,table_name
0,morticd_7_22b4_1642928616
1,morticd_8_9366_1642928616
2,morticd_9_71c9_1642928616
3,morticd_10_1_f695_1642928617
4,morticd_10_2_ffb1_1642928617
5,morticd_10_3_1c6a_1642928618
6,morticd_10_4_453e_1642928618


In [102]:
dataframes = []
for table in df_tables[df_lastupdate_tables]['table_name']:
    if 'morticd' in table:
        firstDash = str(table).index('_', 0)
        secondDash = str(table).index('_', firstDash + 1)
        dataframes.append('df_morticd_' + str(table[firstDash+1:secondDash]))
        # globals()['df_morticd_' + str(table[firstDash+1:secondDash])] = pd.read_sql('SELECT * from ' + config['SCHEMA'] + '.' + table + ';', conn)
    

In [103]:
dataframes

['df_morticd_7',
 'df_morticd_8',
 'df_morticd_9',
 'df_morticd_10',
 'df_morticd_10',
 'df_morticd_10',
 'df_morticd_10']

In [101]:
df_morticd_7

,Country,Admin1,SubDiv,Year,List,Cause,Sex,Frmat,IM_Frmat,Deaths1,...,Deaths21,Deaths22,Deaths23,Deaths24,Deaths25,Deaths26,IM_Deaths1,IM_Deaths2,IM_Deaths3,IM_Deaths4
0,1125,None,None,1955,07A,A000,1,01,08,107822,...,2013.0,2727.0,4126.0,None,None,214,40543,NaN,NaN,NaN
1,1125,None,None,1955,07A,A000,2,01,08,105750,...,1745.0,3465.0,5963.0,None,None,82,39632,NaN,NaN,NaN
2,1125,None,None,1955,07A,A001,1,01,08,1302,...,8.0,6.0,2.0,None,None,1,6,NaN,NaN,NaN
3,1125,None,None,1955,07A,A001,2,01,08,739,...,2.0,1.0,2.0,None,None,0,4,NaN,NaN,NaN
4,1125,None,None,1955,07A,A002,1,01,08,69,...,0.0,0.0,0.0,None,None,0,7,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
281744,4340,None,None,1960,07B,202,2,02,08,4,...,0.0,0.0,0.0,None,None,0,0,NaN,NaN,NaN
281745,4340,None,None,1960,07B,203,1,02,08,13,...,0.0,0.0,0.0,None,None,0,0,NaN,NaN,NaN
281746,4340,None,None,1960,07B,203,2,02,08,13,...,0.0,2.0,0.0,None,None,0,0,NaN,NaN,NaN
281747,4340,None,None,1960,07B,205,1,02,08,0,...,0.0,0.0,0.0,None,None,0,0,NaN,NaN,NaN


In [16]:
# df_morticd = pd.read_sql('SELECT * from formatted.morticd_7_22b4_1642928616;', conn)
# df_countries = pd.read_sql('SELECT * from formatted.countrycodes_0_8c41_1642928619;', conn)

In [114]:
# Check if morticd countries are in countrycode country
for dataframe in dataframes:
    globals()[str(dataframe)]['CountryWrongCode'] = globals()[str(dataframe)]['Country'].apply(lambda x: x in globals()[str(dataframe)].Country)    

0         True
1         True
2         True
3         True
4         True
          ... 
281744    True
281745    True
281746    True
281747    True
281748    True
Name: CountryWrongCode, Length: 281749, dtype: bool
0         True
1         True
2         True
3         True
4         True
          ... 
380622    True
380623    True
380624    True
380625    True
380626    True
Name: CountryWrongCode, Length: 380627, dtype: bool


In [115]:
# Check Size
for dataframe in dataframes[0:1]:
    globals()[str(dataframe)]['CountryWrongSize'] = globals()[str(dataframe)]['Country'].apply(lambda x: x != None and len(str(x))  != 4)
    globals()[str(dataframe)]['Admin1WrongSize'] = globals()[str(dataframe)]['Admin1'].apply(lambda x: x != None and len(str(x)) != 3)
    globals()[str(dataframe)]['SubDivWrongSize'] = globals()[str(dataframe)]['SubDiv'].apply(lambda x: x != None and len(str(x)) != 3)
    globals()[str(dataframe)]['YearWrongSize'] = globals()[str(dataframe)]['Year'].apply(lambda x: x != None and len(str(x)) != 4)
    globals()[str(dataframe)]['ListWrongSize'] = globals()[str(dataframe)]['List'].apply(lambda x: x != None and len(str(x)) != 3)
    globals()[str(dataframe)]['CauseWrongSize'] = globals()[str(dataframe)]['Cause'].apply(lambda x: x != None and len(str(x)) > 4)
    globals()[str(dataframe)]['SexWrongSize'] = globals()[str(dataframe)]['Sex'].apply(lambda x: x != None and len(str(x)) != 1)
    globals()[str(dataframe)]['FrmatWrongSize'] = globals()[str(dataframe)]['Frmat'].apply(lambda x: x != None and len(str(x)) != 2)
    globals()[str(dataframe)]['IM_FrmatWrongSize'] = globals()[str(dataframe)]['IM_Frmat'].apply(lambda x: x != None and len(str(x)) != 2)

In [117]:
for dataframe in dataframes[0:1]:
    death_colnames = globals()[str(dataframe)].filter(regex=("Deaths*")).columns

death_colnames

for death in death_colnames:    
    globals()[str(dataframe)][death + 'WrongSize'] = globals()[str(dataframe)][death].apply(lambda x: len(str(x)) > 8)
    globals()[str(dataframe)][death + 'WrongValue'] = globals()[str(dataframe)][death].apply(lambda x: x >= 0)
   

Index(['Deaths1', 'Deaths2', 'Deaths3', 'Deaths4', 'Deaths5', 'Deaths6',
       'Deaths7', 'Deaths8', 'Deaths9', 'Deaths10', 'Deaths11', 'Deaths12',
       'Deaths13', 'Deaths14', 'Deaths15', 'Deaths16', 'Deaths17', 'Deaths18',
       'Deaths19', 'Deaths20', 'Deaths21', 'Deaths22', 'Deaths23', 'Deaths24',
       'Deaths25', 'Deaths26', 'IM_Deaths1', 'IM_Deaths2', 'IM_Deaths3',
       'IM_Deaths4'],
      dtype='object')

In [47]:
df_morticd[death_colnames]

,Deaths1,Deaths2,Deaths3,Deaths4,Deaths5,Deaths6,Deaths7,Deaths8,Deaths9,Deaths10,...,Deaths21WrongSize,Deaths22WrongSize,Deaths23WrongSize,Deaths24WrongSize,Deaths25WrongSize,Deaths26WrongSize,IM_Deaths1WrongSize,IM_Deaths2WrongSize,IM_Deaths3WrongSize,IM_Deaths4WrongSize
0,107822,40543,18814,8192.0,2860.0,1440.0,2120,1219.0,1059,1133.0,...,False,False,False,False,False,False,False,False,False,False
1,105750,39632,21606,9090.0,3052.0,1404.0,1679,899.0,889,826.0,...,False,False,False,False,False,False,False,False,False,False
2,1302,6,14,17.0,12.0,11.0,26,22.0,85,149.0,...,False,False,False,False,False,False,False,False,False,False
3,739,4,16,13.0,17.0,10.0,28,29.0,75,84.0,...,False,False,False,False,False,False,False,False,False,False
4,69,7,13,4.0,9.0,5.0,13,4.0,2,2.0,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
281744,4,0,0,NaN,NaN,NaN,0,0.0,0,1.0,...,False,False,False,False,False,False,False,False,False,False
281745,13,0,0,NaN,NaN,NaN,0,1.0,0,0.0,...,False,False,False,False,False,False,False,False,False,False
281746,13,0,0,NaN,NaN,NaN,0,0.0,0,0.0,...,False,False,False,False,False,False,False,False,False,False
281747,0,0,0,NaN,NaN,NaN,0,0.0,0,0.0,...,False,False,False,False,False,False,False,False,False,False


In [21]:
wrongs = df_morticd.filter(regex=(".Wrong*")).columns
wrongs

Index(['CountryWrongCode', 'CountryWrongSize', 'Admin1WrongSize',
       'SubDivWrongSize', 'YearWrongSize', 'ListWrongSize', 'CauseWrongSize',
       'SexWrongSize', 'FrmatWrongSize', 'IM_FrmatWrongSize',
       'Deaths1WrongSize', 'Deaths2WrongSize', 'Deaths3WrongSize',
       'Deaths4WrongSize', 'Deaths5WrongSize', 'Deaths6WrongSize',
       'Deaths7WrongSize', 'Deaths8WrongSize', 'Deaths9WrongSize',
       'Deaths10WrongSize', 'Deaths11WrongSize', 'Deaths12WrongSize',
       'Deaths13WrongSize', 'Deaths14WrongSize', 'Deaths15WrongSize',
       'Deaths16WrongSize', 'Deaths17WrongSize', 'Deaths18WrongSize',
       'Deaths19WrongSize', 'Deaths20WrongSize', 'Deaths21WrongSize',
       'Deaths22WrongSize', 'Deaths23WrongSize', 'Deaths24WrongSize',
       'Deaths25WrongSize', 'Deaths26WrongSize', 'IM_Deaths1WrongSize',
       'IM_Deaths2WrongSize', 'IM_Deaths3WrongSize', 'IM_Deaths4WrongSize'],
      dtype='object')

In [22]:
for wrong_col in wrongs:
   if len(df_morticd[df_morticd[wrong_col] == True]) > 0:
      print(wrong_col)

In [ ]:
df_demo_country = pd.read_sql('SELECT * from formatted.DemographicCountry;', conn)
df_demo_labels = pd.read_sql('SELECT * from formatted.DemographicLabels;', conn)
df_demo_indic = pd.read_sql('SELECT * from formatted.Demographic;', conn)

In [ ]:
# Check if demographic country codes are in demographic country table
df_demo_indic['CountryId'] = df_demo_indic['CountryId'].apply(lambda x: x in df_demo_country.Code)

In [ ]:
# Check if demographic Indicator ID are in demographic label table
df_demo_indic['IndicatorId'] = df_demo_indic['Indicator'].apply(lambda x: x in df_demo_labels.Indicator)

In [ ]:
# Check size Demographic label columns
df_demo_labels['IndicatorWrongSize'] = df_demo_labels['Indicator'].apply(lambda x: x != None and len(str(x))  != 50)
df_demo_labels['DescriptionWrongSize'] = df_demo_labels['Description'].apply(lambda x: x != None and len(str(x))  != 100)

In [ ]:
# Check size Demographic country columns
df_demo_country['Code'] = df_demo_country['Code'].apply(lambda x: x != None and len(str(x))  != 3)
df_demo_country['Name'] = df_demo_country['Name'].apply(lambda x: x != None and len(str(x))  != 100)

In [ ]:
# Check size Demographic country columns
df_demo_indic['IndicatorId'] = df_demo_indic['IndicatorId'].apply(lambda x: x != None and len(str(x))  != 50)
df_demo_indic['CountryId'] = df_demo_indic['CountryId'].apply(lambda x: x != None and len(str(x))  != 3)
df_demo_indic['Year'] = df_demo_indic['Year'].apply(lambda x: x != None and len(str(x))  != 4)
df_demo_indic['Magnitude'] = df_demo_indic['Magnitude'].apply(lambda x: x != None and len(str(x))  != 50)
df_demo_indic['Qualifier'] = df_demo_indic['Qualifier'].apply(lambda x: x != None and len(str(x))  != 50)

In [ ]:
# Check Uniques Demographic Country
df_demo_country_unique = pd.DataFrame(df_demo_country['Code'].value_counts())
df_demo_country['CodeWrongUnique'] = df_demo_country_unique['Code'] > 2

In [ ]:
# Check Uniques Demographic Labels
df_demo_labels_unique = pd.DataFrame(df_demo_labels['Indicator'].value_counts())
df_demo_labels['IndicatorWrongUnique'] = df_demo_labels_unique['Indicator'] > 2

In [ ]:
# Check Uniques Country codes
df_country_code_unique = pd.DataFrame(df_countries['Country'].value_counts())
df_countries['CountryWrongUnique'] = df_country_code_unique['Indicator'] > 2

In [ ]:
# Check possitive values
df_morticd['YearWrongValue'] = df_morticd['Year'].apply(lambda x: x != None and x > 0)
df_demo_indic['YearWrongValue'] = df_demo_indic['Year'].apply(lambda x: x != None and x > 0)
df_demo_indic['ValueWrongValue'] = df_demo_indic['Value'].apply(lambda x: x != None and x > 0)